In [1]:
from pathlib import Path
from io import StringIO
import pandas as pd
import numpy as np

In [2]:
grid, moves = Path("data/15.txt").read_text().split("\n\n")
grid = (
    pd.read_csv(StringIO(grid), engine="python", sep=r"\s*", names=range(-1, 1000))
    .dropna(how="all", axis=1)
    .values
)
grid1 = grid.copy()
moves = [x for l in moves.split("\n") for x in l]
dirs = {"<": (0, -1), ">": (0, 1), "^": (-1, 0), "v": (1, 0)}

In [3]:
def find_first(line, ch):
    try:
        i = next(i for i, x in enumerate(line) if x == ch)
    except StopIteration:
        return None
    return i


def pprint(grid):
    print("\n".join("".join(l) for l in grid))

In [4]:
def move(x, y, dir, grid):
    dx, dy = dirs[dir]
    if dir == "<":
        line = grid[x, 1:y][::-1]
    elif dir == ">":
        line = grid[x, (y + 1) : -1]
    elif dir == "^":
        line = grid[1:x, y][::-1]
    elif dir == "v":
        line = grid[(x + 1) : -1, y]

    if (early := find_first(line, "#")) is not None:
        line = line[:early]

    if len(line) == 0:
        return x, y

    i = find_first(line, ".")
    if i is None:
        return x, y

    line[1 : i + 1] = line[:i]
    line[0] = "@"
    grid[x, y] = "."
    return x + dx, y + dy

In [5]:
x, y = np.where(grid == "@")[0][0], np.where(grid == "@")[1][0]
for dir in moves:
    # pprint(grid)
    x, y = move(x, y, dir, grid)
    # print(dir)
# pprint(grid)
sum(i * 100 + j for (i, j), v in np.ndenumerate(grid) if v == "O")

1436690

In [6]:
b = StringIO()
transform = {"#": "##", "O": "[]", ".": "..", "@": "@."}

for line in grid1:
    for x in line:
        b.write(transform[x])
    b.write("\n")
b.seek(0)

wide_grid = (
    pd.read_csv(b, engine="python", sep=r"\s*", names=range(-1, 1000))
    .dropna(how="all", axis=1)
    .values
)

In [7]:
def is_box_movable(x, y, dir, grid):
    dx, _ = dirs[dir]
    if (grid[x + dx, y : y + 2] == np.array([".", "."])).all():
        return True
    if (grid[x + dx, y : y + 2] == np.array([".", "@"])).all():
        return True
    if (grid[x + dx, y : y + 2] == np.array(["@", "."])).all():
        return True
    if "#" in grid[x + dx, y : y + 2]:
        return False
    if (grid[x + dx, y : y + 2] == np.array(["[", "]"])).all():
        return is_box_movable(x + dx, y, dir, grid)
    if (grid[x + dx, y : y + 2] == np.array([".", "["])).all():
        return is_box_movable(x + dx, y + 1, dir, grid)
    if (grid[x + dx, y : y + 2] == np.array(["]", "["])).all():
        return is_box_movable(x + dx, y - 1, dir, grid) and is_box_movable(
            x + dx, y + 1, dir, grid
        )
    if (grid[x + dx, y : y + 2] == np.array(["]", "."])).all():
        return is_box_movable(x + dx, y - 1, dir, grid)
    raise ValueError(grid[x + dx, y : y + 2])


def move_box(x, y, dir, grid):
    dx, _ = dirs[dir]
    if (grid[x + dx, y : y + 2] == np.array(["]", "["])).all():
        move_box(x + dx, y - 1, dir, grid)
        move_box(x + dx, y + 1, dir, grid)
    elif (grid[x + dx, y : y + 2] == np.array([".", "["])).all():
        move_box(x + dx, y + 1, dir, grid)
    elif (grid[x + dx, y : y + 2] == np.array(["]", "."])).all():
        move_box(x + dx, y - 1, dir, grid)
    elif (grid[x + dx, y : y + 2] == np.array(["[", "]"])).all():
        move_box(x + dx, y, dir, grid)

    grid[x + dx, y : y + 2] = np.array(["[", "]"])
    grid[x, y : y + 2] = np.array([".", "."])


def move2(x, y, dir, grid):
    dx, dy = dirs[dir]
    if grid[x + dx, y + dy] == "#":
        return x, y

    if grid[x + dx, y + dy] == ".":
        grid[x + dx, y + dy] = "@"
        grid[x, y] = "."
        return x + dx, y + dy

    if grid[x + dx, y + dy] == "[":
        if is_box_movable(x + dx, y + dy, dir, grid):
            move_box(x + dx, y + dy, dir, grid)
            grid[x + dx, y + dy] = "@"
            grid[x, y] = "."
            return x + dx, y + dy
        else:
            return x, y

    if grid[x + dx, y + dy] == "]":
        if is_box_movable(x + dx, y + dy - 1, dir, grid):
            move_box(x + dx, y + dy - 1, dir, grid)
            grid[x + dx, y + dy] = "@"
            grid[x, y] = "."
            return x + dx, y + dy
        else:
            return x, y

    raise ValueError(grid[x + dx, y + dy])

In [8]:
from copy import deepcopy

x, y = np.where(wide_grid == "@")[0][0], np.where(wide_grid == "@")[1][0]
for dir in moves:
    # pprint(wide_grid)
    if dir == "^" or dir == "v":
        x, y = move2(x, y, dir, wide_grid)
    else:
        x, y = move(x, y, dir, wide_grid)
    # print(dir)
# pprint(wide_grid)

sum(i * 100 + j for (i, j), v in np.ndenumerate(wide_grid) if v == "[")

1482350